In [1]:
#This work was done originally in .py file.
#It has been moved to this ipynb  Jupyter Notebook for consistency purposes.
#[1-4] codes are to demonstrate cleaning steps. 

In [2]:
import pandas as pd
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')
spacex_df.head() #Showing DF

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [3]:
single_df= spacex_df[spacex_df['Launch Site'] == 'KSC LC-39A']
single_df.head() #Single Launch Site

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
36,36,30,KSC LC-39A,Success,1,2490.00,F9 FT B1031.1,FT
37,37,31,KSC LC-39A,Success,0,5600.00,F9 FT B1030,FT
38,38,32,KSC LC-39A,Success,1,5300.00,F9 FT B1021.2,FT
39,39,33,KSC LC-39A,Success,1,3696.65,F9 FT B1032.1,FT
40,40,34,KSC LC-39A,Success,0,6070.00,F9 FT B1034,FT


In [4]:
#Class Count 
df1 = single_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
df1

,Launch Site,class,class count
0,KSC LC-39A,0,3
1,KSC LC-39A,1,10


In [ ]:
#Python Code 

# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout TASK 1
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard for Falcon 9',
                                        style={'textAlign': 'center', 'color': '#000000',
                                               'font-size': 45}),
                                # TASK A: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                    {'label': 'All Sites', 'value': 'All Sites'},
                                                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'}
                                                    ],
                                            placeholder = 'Select a Launch Site',
                                            searchable = True   
                                            ),
                                html.Br(),

                                # TASK B: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK C: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,
                                                max=10000,
                                                step=1000,
                                                value=[min_payload, max_payload]),

                                # TASK D: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value'))
def piechart(site_dropdown):
    df = spacex_df	
    if site_dropdown == 'All Sites':
        fig = px.pie(df, values='class', names='Launch Site', title='Launch Sites Success Rate')
        return fig

    else:
        single_df= spacex_df[spacex_df['Launch Site'] == site_dropdown]
        df1 = single_df.groupby(['Launch Site','class']).size().reset_index(name='class count') #For class count
        title = f"Total Success Launches for site {site_dropdown}"
        fig = px.pie(df1, values='class count', names='class', title=title)
        return fig


# TASK 3:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])
def scatter(site,payload):
    low,high = (payload[0], payload[1])
    df = spacex_df
    ranged_df = df[df['Payload Mass (kg)'].between(low,high)]
    if site == 'All Sites':
        fig = px.scatter(ranged_df,x="Payload Mass (kg)", y="class", color="Booster Version Category", title='Payload VS Outcome (All Sites)')
        return fig
    
    else:
        ranged_df2= ranged_df[ranged_df['Launch Site'] == site]
        fig = px.scatter(ranged_df2,x="Payload Mass (kg)", y="class", color="Booster Version Category", title='Payload VS Outcome for ' + site)
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()
